When use xlm-roberta. Restart the runtime after installing

In [1]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [3]:
import re
import random
import numpy as np
import pandas as pd

import spacy

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer
from transformers import AutoModel, AutoTokenizer

from sklearn.model_selection import train_test_split, StratifiedKFold

from collections import defaultdict

from sklearn.metrics import confusion_matrix, classification_report,precision_score,accuracy_score,f1_score

import pandas as pd
import spacy
import os
import sys

import logging
logging.basicConfig(level = logging.ERROR)

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-3-c14e1bcbde5d>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#model_name = 'bert-base-multilingual-uncased'
model_name = 'bert-base-uncased'
#model_name = 'xlm-roberta-base'

VikiWiki

In [7]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
def load_vikiwiki_dataset(path: str, label: str|int, difficulty: str, lang: str = "en") -> pd.DataFrame:
    cols = ["content", "labels", "difficulty"]
    result = []
    load_path = path
    if os.path.isdir(path):
        dir_file_list = os.listdir(path)
        dir_file_list.sort()
        if lang in dir_file_list:
            load_path = os.path.join(load_path, lang)
            dir_file_list = os.listdir(load_path)
            dir_file_list.sort()
        for file in dir_file_list:
            with open(os.path.join(load_path, file), "r") as f:
                data = "".join(f.readlines())
            if len(data) > 0:
                result.append([data, label, difficulty])
    return pd.DataFrame(
        data=result,
        columns=cols
    )

In [ ]:
df_ca = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "ca"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "ca")
    ],
    axis = 0,
    ignore_index = True
)

df_en = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "en"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "en")
    ],
    axis = 0,
    ignore_index = True
)

df_es = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "es"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "es")
    ],
    axis = 0,
    ignore_index = True
)

df_eu = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "eu"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "eu")
    ],
    axis = 0,
    ignore_index = True
)

df_fr = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "fr"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "fr")
    ],
    axis = 0,
    ignore_index = True
)

df_it = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "it"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "it")
    ],
    axis = 0,
    ignore_index = True
)


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


mean tree height

In [ ]:
!python -m spacy download ca_core_news_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download es_core_news_sm

!python -m spacy download xx_ent_wiki_sm

In [ ]:
def walk_tree(node, depth):
    if node.n_lefts + node.n_rights > 0:
        return max(walk_tree(child, depth + 1) for child in node.children)
    else:
        return depth

def get_mean_tree_height(text):
    depths = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        doc = lang_nlp(sentence)
        depth = [walk_tree(sent.root, 0) for sent in doc.sents]
        depths.extend(depth)
    return np.mean(depths)

In [ ]:
lang_nlp = spacy.load("ca_core_news_sm")
df_ca['mean_tree_height'] = df_ca['content'].progress_apply(get_mean_tree_height)

In [ ]:
lang_nlp = spacy.load('en_core_web_sm')
df_en['mean_tree_height'] = df_en['content'].progress_apply(get_mean_tree_height)

In [ ]:
lang_nlp = spacy.load("es_core_news_sm")
df_es['mean_tree_height'] = df_es['content'].progress_apply(get_mean_tree_height)

In [ ]:
lang_nlp = spacy.load("fr_core_news_sm")
df_fr['mean_tree_height'] = df_fr['content'].progress_apply(get_mean_tree_height)

In [ ]:
lang_nlp = spacy.load("it_core_news_sm")
df_it['mean_tree_height'] = df_it['content'].progress_apply(get_mean_tree_height)

In [ ]:
lang_nlp = spacy.load("xx_ent_wiki_sm")
df_eu['mean_tree_height'] = df_eu['content'].progress_apply(get_mean_tree_height)

In [ ]:
datasets = [df_en, df_es, df_fr, df_it]
langs = ["english", "spanish", "french", "italian"]
for k, df in enumerate(datasets):
  sen_len = []
  for i in df['content']:
    curr_sen_len = []
    for j in sent_tokenize(i, language=langs[k]):
      curr_sen_len.append(len(word_tokenize(j, language=langs[k])))
    mean_len = np.mean(curr_sen_len)
    sen_len.append(mean_len)
  df['mean_sentence_len'] = sen_len

In [ ]:
dataset_mult = [df_ca, df_eu]
for k, df in enumerate(datasets):
  sen_len = []
  for i in df['content']:
    curr_sen_len = []
    for j in sent_tokenize(i):
      curr_sen_len.append(len(word_tokenize(j)))
    mean_len = np.mean(curr_sen_len)
    sen_len.append(mean_len)
  df['mean_sentence_len'] = sen_len

In [ ]:
!pip install lingfeat
from lingfeat import extractor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 47.6 MB/s eta 0:00:00
  Created wheel for lingfeat: filename=lingfeat-1.0.0b19-py3-none-any.whl size=71127048 sha256=a9f6d2a7e22a856032b2c760a893c7aebd9aaca0200ff01b9f78339abcb9ab61
  Stored in directory: /root/.cache/pip/wheels/f0/6c/25/7cc4b4dbfc7c605bad9f8824afef1b4abc8ed18232972c86a8
Successfully built lingfeat


In [ ]:
def get_avg_nounphrase(text):
  LingFeat = extractor.pass_text(text)
  LingFeat.preprocess()
  avg = LingFeat.PhrF_()['as_NoPhr_C']
  return avg

In [ ]:
df_ca['avg_nounphrases'] = df_ca['content'].progress_apply(get_avg_nounphrase)

In [ ]:
df_en['avg_nounphrases'] = df_en['content'].progress_apply(get_avg_nounphrase)

In [ ]:
df_es['avg_nounphrases'] = df_es['content'].progress_apply(get_avg_nounphrase)

In [ ]:
df_eu['avg_nounphrases'] = df_eu['content'].progress_apply(get_avg_nounphrase)

In [ ]:
df_fr['avg_nounphrases'] = df_fr['content'].progress_apply(get_avg_nounphrase)

In [ ]:
df_it['avg_nounphrases'] = df_it['content'].progress_apply(get_avg_nounphrase)

  0%|          | 0/567 [00:00<?, ?it/s]

In [ ]:
df_ca = df_ca.dropna()
df_en = df_en.dropna()
df_es = df_es.dropna()
df_eu = df_eu.dropna()
df_fr = df_fr.dropna()
df_it = df_it.dropna()

In [10]:
root_path = "/content/drive/MyDrive/vikiwiki"

In [11]:
df_ca = pd.read_csv(os.path.join(root_path, "df_ca.csv"))
df_ca

,content,labels,difficulty,mean_sentence_len,mean_tree_height,avg_nounphrases
0,Àfrica és el tercer continent més gran del món...,0,beginner,16.666667,3.000000,4.500000
1,"L'àrab és una llengua semítica, com l'hebreu. ...",0,beginner,13.333333,3.166667,3.166667
2,Àsia és el continent més gran i poblat de la T...,0,beginner,20.000000,3.000000,1.250000
3,Ígor Stravinski (1882 - 1971) va ser un compos...,0,beginner,20.333333,4.000000,2.800000
4,Úmbria (en italià: Umbria) és una regió d'Itàl...,0,beginner,9.000000,2.666667,2.000000
...,...,...,...,...,...,...
850,"Xile, oficialment la República de Xile (Repúbl...",1,advanced,27.313492,4.573585,5.394737
851,"Yaren, abans anomenat Makwa, és un districte i...",1,advanced,19.000000,3.444444,4.000000
852,Iuri Aleksèievitx Gagarin (en rus: Ю́рий Алекс...,1,advanced,26.745098,4.564815,6.192661
853,Zàmbia és un estat de l'Àfrica Austral que lim...,1,advanced,24.548673,4.622807,5.428571


In [12]:
df_en = pd.read_csv(os.path.join(root_path, "df_en.csv"))
df_en

,content,labels,difficulty,mean_sentence_len,avg_nounphrases,mean_tree_height
0,"The United States dollar (sign: $), also calle...",0,beginner,38.200000,7.000000,7.800000
1,Édouard Manet (23 January 1832 – 30 April 1883...,0,beginner,16.333333,3.500000,5.000000
2,Pi or π may refer to:\n,0,beginner,7.000000,0.500000,2.000000
3,The 1800s were a time full of advancements in ...,0,beginner,11.937500,3.555556,4.470588
4,2013 was a common year. It was followed by 201...,0,beginner,8.000000,0.666667,2.500000
...,...,...,...,...,...,...
859,"World War II (WWII or WW2), also known as the ...",1,advanced,30.316929,12.615804,5.778598
860,The Xbox 360 is a home video game console deve...,1,advanced,30.538462,10.873418,5.937220
861,The Yangtze River (English pronunciation: /ˈjæ...,1,advanced,26.546366,9.205634,5.746377
862,"Yugoslavia (Serbo-Croatian, Macedonian, Sloven...",1,advanced,26.599222,9.380165,5.942085


In [13]:
df_es = pd.read_csv(os.path.join(root_path, "df_es.csv"))
df_es

,content,labels,difficulty,mean_sentence_len,mean_tree_height,avg_nounphrases
0,tk es dominio de primer nivel para Tokelau.\n,0,beginner,9.000000,3.000000,2.000000
1,"Doce (lat. ""duodecim"", dos más diez) es un núm...",0,beginner,15.500000,3.000000,2.400000
2,Trece es un número natural entre el doce y el ...,0,beginner,13.333333,4.333333,2.750000
3,Quince es un número natural entre el trece y e...,0,beginner,14.333333,3.333333,2.500000
4,1955 fue un año del siglo XX de 365 días.\n,0,beginner,11.000000,2.000000,1.000000
...,...,...,...,...,...,...
827,El yo-yo es un juguete formado por un disco de...,1,advanced,24.960000,4.461538,5.909091
828,Zacatecas es uno de los treinta y un estados q...,1,advanced,33.501370,5.176781,6.524457
829,Zagreb escucha (?·i) es la capital y la ciuda...,1,advanced,24.462500,4.172840,5.325000
830,"Daucus carota subespecie sativus, llamada popu...",1,advanced,28.288889,4.604167,6.255814


In [14]:
df_eu = pd.read_csv(os.path.join(root_path, "df_eu.csv"))
df_eu

,content,labels,difficulty,mean_sentence_len,avg_nounphrases,mean_tree_height
0,"Ameriketako Estatu Batuak (laburtuta, AEB) Ame...",0,beginner,12.000000,2.600000,4.333333
1,"Abaraska erleek (baita liztor batzuek ere), ez...",0,beginner,19.666667,2.500000,4.333333
2,Abeltzaintza edo abelazkuntza etxe-abereak haz...,0,beginner,16.600000,3.000000,3.416667
3,"Abeltzaintza estentsiboan, ganadua elikatzeko,...",0,beginner,16.000000,1.666667,4.500000
4,"Aberea animalia ugaztun nagusia da, piztia ez ...",0,beginner,20.333333,3.500000,5.333333
...,...,...,...,...,...,...
814,"Zientzialari, zentzu zabalean, jakintza lortze...",1,advanced,14.600000,3.272727,4.000000
815,Zigilua (edo seilua[1]) posta bidez bidali beh...,1,advanced,17.318182,3.592593,3.884615
816,"Zimbabwe[1], Afrikako hegoaldeko estatua, itsa...",1,advanced,16.968750,3.189944,3.652174
817,"Zuberoa (zubereraz, Xiberua edo Xiberoa; frant...",1,advanced,17.347222,3.320175,3.591837


In [15]:
df_fr = pd.read_csv(os.path.join(root_path, "df_fr.csv"))
df_fr

,content,labels,difficulty,mean_sentence_len,mean_tree_height,avg_nounphrases
0,Le copyright désigne le droit de l'auteur ou d...,0,beginner,17.750000,4.500000,3.200000
1,L'Édit de Milan est un ensemble de mesures pri...,0,beginner,19.000000,3.615385,3.750000
2,L'élection présidentielle française de 2002 es...,0,beginner,19.750000,4.133333,4.062500
3,Le cerveau est un organe situé dans le crâne e...,0,beginner,21.485714,3.837209,4.166667
4,"Élisabeth II, née Elizabeth Alexandra Mary le ...",0,beginner,22.111111,3.153846,3.583333
...,...,...,...,...,...,...
751,Versailles (prononcer [vɛʁ.sɑj]) est une commu...,1,advanced,30.472362,4.395538,4.484018
752,Si vous disposez d'ouvrages ou d'articles de r...,1,advanced,32.539683,4.972028,5.257576
753,"En Belgique, une ville est une commune portant...",1,advanced,25.000000,5.230769,3.666667
754,Windows Phone est un système d'exploitation mo...,1,advanced,27.508591,4.271739,4.428571


In [16]:
df_it = pd.read_csv(os.path.join(root_path, "df_it.csv"))
df_it

,content,labels,difficulty,mean_sentence_len,avg_nounphrases,mean_tree_height
0,Água Boa è un comune di 18.994 abitanti (2007)...,0,beginner,19.000000,2.000000,4.000000
1,.bf è il dominio di primo livello nazionale (c...,0,beginner,15.000000,1.333333,3.000000
2,.br è il dominio di primo livello nazionale (c...,0,beginner,39.000000,3.333333,6.000000
3,.mx è il dominio di primo livello nazionale (c...,0,beginner,15.000000,1.333333,3.000000
4,.ru è il dominio di primo livello nazionale (c...,0,beginner,15.000000,2.000000,1.500000
...,...,...,...,...,...,...
835,"Vittorio Gassman, nato Gassmann (Struppa, 1º s...",1,advanced,33.535211,4.869565,4.505747
836,Wonkifong è un comune (in francese sous-préfec...,1,advanced,23.000000,3.000000,6.000000
837,Yosemite Sam (tradotto in alcuni cartoni come ...,1,advanced,23.315789,5.150000,4.000000
838,"Arthur Antunes Coimbra, meglio noto come Zico,...",1,advanced,28.666667,4.970588,4.684211


Dataset

In [17]:
df = df_ca

In [18]:
num_diff = df.labels.unique()
diff = ['beginner', 'advanced']

label = dict(zip(num_diff, diff))

diff = list(diff)


In [19]:
class DiffDataset(Dataset):
    def __init__(self, contents, labels, tokenizer, max_len, feature_1=None, feature_2=None, feature_3=None):
        self.contents = contents
        self.labels = labels
        self.tokenizer = tokenizer

        self.feature_1 = feature_1
        self.feature_2 = feature_2
        self.feature_3 = feature_3

        self.max_len = max_len

    def __len__(self):
        return len(self.contents)

    def __getitem__(self, item):
        content = str(self.contents[item])
        labels = self.labels[item]

        feature_1 = self.feature_1[item]
        feature_2 = self.feature_2[item]
        feature_3 = self.feature_3[item]

        # Tokenizing the texts, while also including special tokens
        # for start and end of the text, as well as padding
        encoding = self.tokenizer.encode_plus(
          content,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt', # We return here the data as Pytorch Tensor
        )

        return {
          'content': content,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(labels, dtype=torch.long),
          'feature_1': torch.tensor([feature_1], dtype=torch.long),
          'feature_2': torch.tensor([feature_2], dtype=torch.long),
          'feature_3': torch.tensor([feature_3], dtype=torch.long)
        }


In [20]:
# df_temp, df_test = train_test_split(dataset, test_size=0.2, random_state=42)

# print("size of test set : " + str(df_test.shape[0]))

In [21]:
def generate_dataloader(df, tokenizer, max_len, batch_size):
    ds = DiffDataset(
        contents=df.content.to_numpy(),
        labels=df.labels.to_numpy(),

        feature_1 = df.mean_sentence_len.to_numpy(),
        feature_2 = df.mean_tree_height.to_numpy(),
        feature_3 = df.avg_nounphrases.to_numpy(),


        tokenizer=tokenizer,
        max_len=max_len,
      )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
      )


In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [23]:
max_len = 128
batch_size = 8

# train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
# val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
# test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)

# data = next(iter(train_dataloader))



In [24]:
class DiffClassifier(nn.Module):

    def __init__(self, n_classes):
        super(DiffClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        # Adding drop out, keeping 90% of the last neurons of the raw BERT model
        #self.drop = nn.Dropout(p=0.1)
        # The last linear layer for multiclass classification
        self.out = nn.Linear(self.bert.config.hidden_size+3, n_classes)

    # Forward propagation function
    def forward(self, input_ids, attention_mask, feature_1, feature_2, feature_3):
        model_outs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = model_outs[0]

        pooled_output = model_outs[1]
        pooled_output = torch.cat((pooled_output, feature_1, feature_2, feature_3) , dim=1)

        return self.out(pooled_output)


In [25]:
# Computing multiclass precision for the outputs of the model
def compute_precision(outputs, labels):
  op = outputs.cpu()
  la = labels.cpu()
  _, preds = torch.max(op, dim=1)
  # We choose 'weighted' averaging of the precision of each label because it takes into account the imbalance of labels in our tweets dataset
  # other viable averaging methods are 'micro'
  return torch.tensor(precision_score(la, preds, average='weighted',zero_division=0))

In [26]:
def train(model, train_dataloader,optimizer,scheduler, loss_fn, df_train):

        # Reset tracking variables at the beginning of each epoch
        precision, correct_predictions, batch_counts = 0, 0, 0
        losses = []

        # Put the model into the training mode
        model.train()


        # For each batch of training data...
        for d in train_dataloader:
              batch_counts +=1
              # Load batch to GPU
              input_ids = d["input_ids"].to(device)
              attention_mask = d["attention_mask"].to(device)
              labels = d["labels"].to(device)
              feature_1 = d["feature_1"].to(device)
              feature_2 = d["feature_2"].to(device)
              feature_3 = d["feature_3"].to(device)

              outputs = model(input_ids=input_ids,attention_mask=attention_mask, feature_1=feature_1, feature_2=feature_2, feature_3=feature_3)

              _, preds = torch.max(outputs, dim=1)

              # Compute loss and accumulate the loss values

              loss = loss_fn(outputs, labels)

              correct_predictions += torch.sum(preds == labels)
              losses.append(loss.item())
              precision +=  compute_precision(outputs, labels)

              loss.backward()

              # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
              # Update parameters and the learning rate
              optimizer.step()
              scheduler.step()
              optimizer.zero_grad()
          # Accuracy, loss, precision
        return correct_predictions.double() / len(df_train), np.mean(losses), precision/batch_counts

In [27]:
def eval(model, valid_dataloader, loss_fn, device, n):
    model = model.eval()


    correct_predictions , precision ,batch_counts = 0,0,0
    losses = []

    with torch.no_grad():
        for d in valid_dataloader:
            batch_counts += 1

            # Preparing inputs
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            feature_1 = d["feature_1"].to(device)
            feature_2 = d["feature_2"].to(device)
            feature_3 = d["feature_3"].to(device)

            # Running inference using the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                feature_1=feature_1,
                feature_2=feature_2,
                feature_3=feature_3
              )

            # Running softmax on the outputs
            _, preds = torch.max(outputs, dim=1)

            # Computing loss function
            loss = loss_fn(outputs, labels)

            # Counting the correct occurences
            correct_predictions += torch.sum(preds == labels)

            # Computing the precision (true positives/true positives + false positives)
            # for each class and label, and find their average weighted by support
            precision += compute_precision(outputs,labels)

            losses.append(loss.item())
    # Accuracy, loss, precision
    return correct_predictions.double()/n, np.mean(losses), precision/batch_counts

In [28]:
def testing(model, dataloader):
  model = model.eval()

  contents = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in dataloader:

      texts = d["content"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["labels"].to(device)
      feature_1 = d["feature_1"].to(device)
      feature_2 = d["feature_2"].to(device)
      feature_3 = d["feature_3"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        feature_1=feature_1,
        feature_2=feature_2,
        feature_3=feature_3
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      contents.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return contents, predictions, prediction_probs, real_values

In [29]:
best_model_path = "./bert_model_baseline.bin"
def save_best_model(model):
  torch.save(model.state_dict(), best_model_path)

def load_best_model(model):
  model.load_state_dict(torch.load(best_model_path))
  return model

In [30]:
class EarlyStopping:
    def __init__(self, model, patience=1):
        self.patience = patience  # number of times to allow for no improvement before stopping the execution
        self.counter = 0  # count the number of times the validation accuracy not improving
        self.min_validation_loss = np.inf
        self.model = model
        self.max_acc = 0

    # return True when validation loss is not decreased by the `min_delta` for `patience` times
    def early_stop_check(self, val_acc):
        if (val_acc > self.max_acc):
            self.max_acc = val_acc
            self.counter = 0  # reset the counter if validation loss decreased at least by min_delta
            save_best_model(self.model)
        else:
            self.counter += 1 # increase the counter if validation loss is not decreased by the min_delta
            if self.counter >= self.patience:
                return True
        return False


In [31]:
def train_loop(df):

  b_accuracy = 0
  epochs = 10
  n_fold = 10
  random_state = 42
  folds_acc = []
  histories = []


  k_fold = StratifiedKFold(n_splits=n_fold, random_state=random_state, shuffle=True)
  y = df.drop(["content", "difficulty", "mean_tree_height", "mean_sentence_len", "avg_nounphrases"], axis=1)

  for i, (temp_index, test_index) in enumerate(k_fold.split(df, y)):
    print(" ------   Fold {}  ------- ".format(i+1), end="\n")
    df_temp = df.iloc[temp_index]
    df_train, df_val = train_test_split(df_temp, test_size=0.2, random_state=42)
    df_test = df.iloc[test_index]
    train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
    val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
    test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)



    data = next(iter(train_dataloader))
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    feature_1 = data['feature_1'].to(device)
    feature_2 = data['feature_2'].to(device)
    feature_3 = data['feature_3'].to(device)

    model = DiffClassifier(len(diff))
    # Running the classifier on GPU
    model = model.to(device)

    optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  weight_decay = 0.2,
                  correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0.1,
                                                num_training_steps=len(train_dataloader)*epochs)
    loss_fn = nn.CrossEntropyLoss().to(device)

    history = defaultdict(list)

    early_stopping = EarlyStopping(model, patience=4)

    for epoch in range(epochs):

      print(f'Epoch {epoch + 1}/{epochs}')
      print('-' * 20)

      train_acc, train_loss, train_preci = train(
            model,
            train_dataloader,
            optimizer,
            scheduler,
            loss_fn,
            df_train
          )
      print(f"Train : Loss {train_loss}, Accuracy : {train_acc*100:.2f} %, Precision : {train_preci}")

      history['train_acc'].append(train_acc)
      history['train_loss'].append(train_loss)
      history['train_precision'].append(train_preci)

      val_acc, val_loss, val_preci = eval(
            model,
            val_dataloader,
            loss_fn,
            device,
            len(df_val),
          )

      print(f'Val : Loss :{val_loss}, Accuracy : {val_acc*100:.2f} %, Precision : {val_preci}')
      print()

      history['val_acc'].append(val_acc)
      history['val_loss'].append(val_loss)
      history['val_precision'].append(val_preci)

      histories.append(history)

      if early_stopping.early_stop_check(val_acc):
          break

        # if val_acc > b_accuracy:
        #   torch.save(model.state_dict(), best_model_path)
        #   b_accuracy = val_acc

    model = load_best_model(model)
    y_contents, y_pred, y_pred_probs, y_test = testing(
          model,
          test_dataloader
        )
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy for fold {}: {:.2f}%\n".format(i+1, acc*100))
    folds_acc.append(acc)
  return folds_acc, histories

Basque Language

In [32]:
df = df_eu

In [ ]:
folds_acc = train_loop(df)

In [36]:
folds_acc[0]

[0.6707317073170732,
 0.6341463414634146,
 0.6707317073170732,
 0.6585365853658537,
 0.524390243902439,
 0.7560975609756098,
 0.7682926829268293,
 0.5609756097560976,
 0.7317073170731707,
 0.7407407407407407]

In [37]:
print("The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Basque language is: {:.3f}%".format(sum(folds_acc[0]) * 100 / len(folds_acc[0])))

The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Basque language is: 67.164%


Catalan Language

In [ ]:
df = df_ca

In [ ]:
folds_acc = train_loop(df)

In [ ]:
folds_acc, train_histories = folds_acc[0], folds_acc[1]

In [ ]:
print("The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Catalan language is: {:.3f}%".format(sum(folds_acc) * 100 / len(folds_acc)))

The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Catalan language is: 94.617%


English Language

In [ ]:
df = df_en

In [ ]:
folds_acc, train_histories = train_loop(df)

In [ ]:
print("The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for English language is: {:.3f}%".format(sum(folds_acc) * 100 / len(folds_acc)))

The accuracy score is: 91.698%


French Language

In [38]:
df = df_fr

In [ ]:
folds_acc, train_histories = train_loop(df)

In [40]:
print("The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for French language is: {:.3f}%".format(sum(folds_acc) * 100 / len(folds_acc)))

The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for French language is: 86.514%


Italian Language

In [ ]:
df = df_it

In [ ]:
folds_acc, train_histories = train_loop(df)

In [ ]:
print("The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Italian language is: {:.3f}%".format(sum(folds_acc) * 100 / len(folds_acc)))

The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Italian language is: 84.881%


Spanish Language

In [41]:
df = df_es

In [ ]:
folds_acc, train_histories = train_loop(df)

In [43]:
print("The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Spanish language is: {:.3f}%".format(sum(folds_acc) * 100 / len(folds_acc)))

The accuracy score for mean_tree_height, mean_sentence_len and avg_nounphrases for Spanish language is: 82.803%
